# Jリーグ観客動員数予測コンテスト　特徴量エンジニアリング

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import plotly.plotly as py
import plotly.graph_objs as go
import plotly
plotly.offline.init_notebook_mode(connected=True)
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from pandas.tseries.offsets import *
%matplotlib inline

### データ読込

In [2]:
train = pd.read_csv("train.csv" )
test = pd.read_csv("test.csv") 
holidays_in_japan = pd.read_csv("holidays_in_japan.csv" )
match_reports= pd.read_csv("match_reports.csv" )
stadium_capacity_mapping= pd.read_csv("stadium_capacity_mapping.csv" )

#### 外部データ読込

In [3]:
ex_stadium_capacity_mapping= pd.read_csv("ex_stadium_capacity_mapping.csv" , encoding="shift_jis")
ex_compose_reports= pd.read_csv("ex_total.csv" , encoding="shift_jis")

In [4]:
ex_match_reports= pd.read_csv("ex_match_reports_mini.csv")

In [5]:
ex_compose_reports.shape

(14424, 14)

In [6]:
ex_compose_reports.columns

Index(['attendance', 'away_team', 'broadcasters', 'division', 'home_team',
       'id', 'kick_off_time', 'match_date', 'round', 'section', 'humidity',
       'temperature', 'venue', 'weather'],
      dtype='object')

In [7]:
train.columns

Index(['id', 'match_date', 'kick_off_time', 'section', 'round', 'home_team',
       'away_team', 'venue', 'weather', 'temperature', 'humidity',
       'broadcasters', 'attendance'],
      dtype='object')

divisionが不明　データ点数はほぼ同じ　二つに分けているだけか？

In [8]:
ex_compose_reports["division"].value_counts()

1    7350
2    7074
Name: division, dtype: int64

#### 外部データのdivisionは不明・かつ,train/testにないので削除 

In [9]:
ex_compose_reports = ex_compose_reports.drop("division" , axis=1)

### 外部データから抽出・昔のデータ取得する

In [10]:
ex_compose_reports =  ex_compose_reports.loc[(pd.to_datetime(ex_compose_reports["match_date"]) < pd.to_datetime("2006-3-31")),:]

## データをまとめていじれるようにtrain,testデータを統合

In [11]:
compose = pd.concat((ex_compose_reports,train,test),axis=0)
#compose = pd.concat((train,test),axis=0)
compose = compose.reset_index().copy() #indexコピー
print(train.shape)
print(test.shape)
print(ex_compose_reports.shape)
print(compose.shape)

(3366, 13)
(477, 12)
(5167, 13)
(9010, 14)


In [12]:
#重複行削除
ex_compose_reports2= ex_compose_reports.drop_duplicates(inplace = False,subset="id")

#重複行削除
train2= train.drop_duplicates(inplace = False,subset="id")

#重複行削除
test2= test.drop_duplicates(inplace = False,subset="id")

In [13]:
print(train2.shape)
print(test2.shape)
print(ex_compose_reports2.shape)
print(compose.shape)

(3366, 13)
(477, 12)
(5167, 13)
(9010, 14)


In [14]:
train.columns

Index(['id', 'match_date', 'kick_off_time', 'section', 'round', 'home_team',
       'away_team', 'venue', 'weather', 'temperature', 'humidity',
       'broadcasters', 'attendance'],
      dtype='object')

In [15]:
print(ex_compose_reports2.columns)

Index(['attendance', 'away_team', 'broadcasters', 'home_team', 'id',
       'kick_off_time', 'match_date', 'round', 'section', 'humidity',
       'temperature', 'venue', 'weather'],
      dtype='object')


### 欠損データの確認

In [16]:
# 欠損データの確認 missing_values_tableを作る
def missing_values_table(df):
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum()/len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table_ren_columns = mis_val_table.rename(
    columns= {0 : 'Missing Values', 1: '% of total values'})
    return mis_val_table_ren_columns

In [17]:
missing_values_table(compose)

,Missing Values,% of total values
index,0,0.000000
attendance,477,5.294118
away_team,0,0.000000
broadcasters,1765,19.589345
home_team,0,0.000000
humidity,198,2.197558
id,0,0.000000
kick_off_time,0,0.000000
match_date,0,0.000000
round,0,0.000000


2018年　第33節　第34節の天気系がデータがない
→そもそも、この部分は予測する部分なのでないよね。
年によって違うが傾向は似ているので、それで対応する？

broadcastのnanデータは、そもそも放送していない

### attendanceは今回予測するデータ 他に気温・湿度・天候にわずかながらNanデータが存在する
なので,欠損データは今回平均で置き換えることにした　https://newtechnologylifestyle.net/pandasnan/
精査必要あり（10/21）

In [18]:
#欠損値を平均で置き換える
compose["humidity"]    = compose["humidity"].fillna(compose["humidity"].mean())
compose["temperature"] = compose["temperature"].fillna(compose["temperature"].mean())
#compose_processing["weather"]   = compose_processing["weather"].fillna(compose_processing["weather"].mean())

In [19]:
holidays_in_japan 
#日本の祝日　祝日は来場者が多いと予想される。　フラグをたてる事を考える

,holiday_date,description
0,2006-01-01,New Years Day
1,2006-01-02,New Years Holiday
2,2006-01-03,New Years Holiday
3,2006-01-09,Coming-of-age Day
4,2006-02-11,National Foundation Day
5,2006-03-21,Vernal Equinox Day
6,2006-04-29,Greenery Day
7,2006-05-03,Constitution Memorial Day
8,2006-05-04,National Holiday
9,2006-05-05,Childrens Day


In [20]:
match_reports  .head(3)
#人気選手がいるかもしれない...
#データ数が多くなるのでとりあえず保留

,id,home_team_player11,home_team_player10,home_team_player9,home_team_player8,home_team_player7,home_team_player6,home_team_player5,home_team_player4,home_team_player3,...,away_team_player2,away_team_player3,away_team_player4,away_team_player5,away_team_player6,away_team_player7,away_team_player8,away_team_player9,away_team_player10,away_team_player11
0,9190,9 マグノ アウベス FW,8 フェルナンジーニョ FW,10 二川 孝広 MF,7 遠藤 保仁 MF,27 橋本 英郎 MF,17 明神 智和 MF,14 家長 昭博 DF,6 山口 智 DF,2 シジクレイ DF,...,2 坪井 慶介 DF,4 田中 マルクス闘莉王 DF,20 堀之内 聖 DF,6 山田 暢久 MF,17 長谷部 誠 MF,13 鈴木 啓太 MF,8 三都主 アレサンドロ MF,18 小野 伸二 MF,10 ポンテ FW,21 ワシントン FW
1,9191,18 長谷川 太郎 FW,16 バレー FW,11 宇留野 純 FW,31 林 健太郎 MF,25 鈴木 健太 MF,8 倉貫 一毅 MF,4 山本 英臣 DF,20 ビジュ DF,15 アライール DF,...,25 市川 大祐 DF,26 青山 直晃 DF,4 高木 和道 DF,3 山西 尊裕 DF,13 兵働 昭弘 MF,7 伊東 輝悦 MF,16 枝村 匠馬 MF,10 藤本 淳吾 MF,18 チョ ジェジン FW,17 マルキーニョス FW
2,9192,9 ルーカス FW,21 ササ サルセード FW,35 リチェーリ FW,6 今野 泰幸 MF,23 梶山 陽平 MF,19 伊野波 雅彦 MF,15 鈴木 規郎 DF,5 増嶋 竜也 DF,2 茂庭 照幸 DF,...,2 三木 隆司 DF,4 深谷 友基 DF,22 上本 大海 DF,5 エジミウソン MF,11 トゥーリオ MF,6 梅田 高志 MF,17 根本 裕一 MF,26 内村 圭宏 MF,13 高松 大樹 FW,9 オズマール FW


In [21]:
stadium_capacity_mapping  .head(3)
#使用されていないスタジアムもあるが、capacityのデータは必須だと考える

,stadium,capacity
0,日産スタジアム,72081
1,埼玉スタジアム2002,62010
2,国立競技場,54224


## スタジアムデータ統合

In [22]:
ex_stadium_capacity_mapping.columns = ["stadium","capacity"]
compose_stadium_capacity_mapping = pd.concat((ex_stadium_capacity_mapping,stadium_capacity_mapping),axis=0)
compose_stadium_capacity_mapping = compose_stadium_capacity_mapping.reset_index().copy() #indexコピー
print(compose_stadium_capacity_mapping.shape)

(265, 3)


In [23]:
print(compose_stadium_capacity_mapping["stadium"].value_counts())
print(len(compose_stadium_capacity_mapping["stadium"].value_counts()))
#本来の要素は190ある

鳴門・大塚スポーツパークポカリスエットスタジアム    2
国立競技場                       2
栃木県グリーンスタジアム                2
維新みらいふスタジアム                 2
三協フロンテア柏スタジアム               2
山梨中銀スタジアム                   2
京都市西京極総合運動公園陸上競技場兼球技場       2
ニッパツ三ツ沢球技場                  2
横浜市三ツ沢公園球技場                 2
新潟スタジアム                     2
熊谷スポーツ文化公園陸上競技場             2
フクダ電子アリーナ                   2
さいたま市浦和駒場スタジアム              2
等々力陸上競技場                    2
ベストアメニティスタジアム               2
長崎市総合運動公園かきどまり陸上競技場         2
エコパスタジアム                    2
ノエビアスタジアム神戸                 2
味の素スタジアム                    2
ヤンマースタジアム長居                 2
熊本県民総合運動公園陸上競技場             2
岐阜メモリアルセンター長良川競技場           2
鹿児島県立鴨池陸上競技場                2
トランスコスモススタジアム長崎             2
日本平スタジアム                    2
レベルファイブスタジアム                2
パロマ瑞穂スタジアム                  2
県立カシマサッカースタジアム              2
神戸総合運動公園ユニバー記念競技場           2
石川県西部緑地公園陸上競技場              2
                           ..
ミクニワールドスタジアム北九州             1
ＳｈｏｎａｎＢＭＷスタジアム平塚            1
Pikaraスタジア

スタジアムのデータを統合したが、重複しているデータが存在してる  
重複データを削除して,indexもリセットする

In [24]:
#邪魔な列を削除する
compose_stadium_capacity_mapping = compose_stadium_capacity_mapping.drop("index" , axis=1)

In [25]:
compose_stadium_capacity_mapping.shape

(265, 2)

In [26]:
#重複行削除
compose_stadium_capacity_mapping.drop_duplicates(inplace = True,subset="stadium")

In [27]:
compose_stadium_capacity_mapping.shape

(190, 2)

In [28]:
#indexリセット
compose_stadium_capacity_mapping = compose_stadium_capacity_mapping.reset_index()

In [29]:
#重複削除 stadiumのnanデータ削除
compose_stadium_capacity_mapping =  compose_stadium_capacity_mapping.drop_duplicates().dropna(subset=['stadium',"capacity"])

In [30]:
compose_stadium_capacity_mapping = compose_stadium_capacity_mapping.drop("index" , axis=1)

In [31]:
print(compose_stadium_capacity_mapping["stadium"].value_counts())
print(len(compose_stadium_capacity_mapping["stadium"].value_counts()))

鳴門・大塚スポーツパークポカリスエットスタジアム    1
佐伯市営陸上競技場                   1
愛鷹広域公園多目的競技場                1
ニンジニアスタジアム                  1
埼玉スタジアム2002                 1
岩手県営陸上競技場                   1
江東区夢の島競技場                   1
うまかな・よかなスタジアム               1
下関市営下関陸上競技場                 1
大垣市浅中公園総合グラウンド陸上競技場         1
千代台公園陸上競技場                  1
江戸川区陸上競技場                   1
パロマ瑞穂スタジアム                  1
瑞穂公園陸上競技場                   1
キンチョウスタジアム                  1
駒沢陸上競技場                     1
ジュビロ磐田サッカースタジアム             1
函館市千代台公園陸上競技場               1
横浜市三ツ沢公園陸上競技場               1
埼玉スタジアム２００２                 1
白波スタジアム                     1
大分銀行ドーム                     1
レベルファイブスタジアム                1
駒沢オリンピック公園総合運動場陸上競技場        1
ニッパツ三ツ沢球技場                  1
維新百年記念公園陸上競技場               1
京都市西京極総合運動公園陸上競技場兼球技場       1
近鉄花園ラグビー場                   1
さいたま市大宮公園サッカー場              1
三協フロンテア柏スタジアム               1
                           ..
ベストアメニティスタジアム               1
沖縄県総合運動公園陸上競技場              1
沖縄市陸上競技場  

## 試合結果統合

In [32]:
#それぞれの配列に列追加
match_reports["home_team_win"] = match_reports["home_team_score"] > match_reports["away_team_score"]
ex_match_reports["home_team_win"] = ex_match_reports["home_team_score"] > ex_match_reports["away_team_score"]

In [33]:
#必要な列だけ抽出
match_reports_mini = match_reports.loc[:,['id',"home_team_win", "home_team_score", "away_team_score"]]
compose_match_reports = pd.concat((ex_match_reports,match_reports_mini),axis=0)
compose_match_reports = compose_match_reports.reset_index().copy() #indexコピー
compose_match_reports = compose_match_reports.drop("index",axis=1)
print(compose_match_reports.shape)

(18201, 4)


In [34]:
#重複行削除
compose_match_reports2 = compose_match_reports.drop_duplicates(inplace = False,subset="id")

In [35]:
compose_match_reports2.shape

(14376, 4)

In [36]:
compose_match_reports2["id"] = compose_match_reports2["id"].astype(str)

C:\Users\Mustang8\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



## 特徴量作成・数値変換

In [37]:
compose_processing = compose.copy() #加工データ用にcopy

In [38]:
compose_processing.shape

(9010, 14)

In [39]:
#ユニーク数カウント composeのデータフレームのその列にどのくらい要素が入っているのか
at_number =compose_processing["away_team"].drop_duplicates().nunique()
broadcasters_number = compose_processing["broadcasters"].drop_duplicates().nunique()+1#外部データありのとき＋１する nanデータ対応
ht_number = compose_processing["home_team"].drop_duplicates().nunique()
humidity_number = compose_processing["humidity"].drop_duplicates().nunique()+1


round_number = compose_processing["round"].drop_duplicates().nunique()
section_number = compose_processing["section"].drop_duplicates().nunique()

temp_number = compose_processing["temperature"].drop_duplicates().nunique() +1#外部データありのとき＋１する nanデータ対応
venue_number = compose_processing["venue"].drop_duplicates().nunique()#外部データありのとき＋１する nanデータ対応
weather_number = compose_processing["weather"].drop_duplicates().nunique()+1#外部データありのとき＋１する nanデータ対応

#match_date_number = compose["match_date"].drop_duplicates().nunique()

In [40]:
#アウェイチーム名の辞書作成
away_team_dict={}
for away_team_num in range(at_number):
    away_team_dict.update({compose["away_team"].drop_duplicates().unique()[away_team_num]:away_team_num})
#置換
compose_processing["away_team"] =compose_processing["away_team"].replace(away_team_dict)

In [41]:
#放送会社の辞書の作成
broadcasters_dict={}
for broadcasters_num in range (broadcasters_number):
    broadcasters_dict.update({compose["broadcasters"].drop_duplicates().unique()[broadcasters_num]:broadcasters_num})
#置換
compose_processing["broadcasters"] =compose_processing["broadcasters"].replace(broadcasters_dict)

In [42]:
compose_processing.shape

(9010, 14)

In [43]:
#ホームチーム名の辞書作成
home_team_dict={}
for home_team_num in range(ht_number):
    home_team_dict.update({compose["home_team"].drop_duplicates().unique()[home_team_num]:home_team_num})
#置換
compose_processing["home_team"] =compose_processing["home_team"].replace(home_team_dict)

In [44]:
compose_processing.shape

(9010, 14)

In [45]:
#ラウンド辞書作成
round_dict={}
for round_num in range(round_number):
    round_dict.update({compose["round"].drop_duplicates().unique()[round_num]:round_num})
#置換
compose_processing["round"] =compose_processing["round"].replace(round_dict)

In [46]:
compose_processing.shape

(9010, 14)

In [47]:
#第～節の辞書作成
section_dict ={}
for section_num in range(section_number):
    section_dict.update({compose["section"].drop_duplicates().unique()[section_num]:section_num})
#置換
compose_processing["section"] =compose_processing["section"].replace(section_dict)

In [48]:
compose_processing.shape

(9010, 14)

下が外部データを追加したモノ

In [49]:
#tcompose_processingにcapacityを追加するため
#mergeするために,列名を同じ名前にする .rename
compose_stadium_capacity_mapping = compose_stadium_capacity_mapping.rename(columns={"stadium":"venue"})

In [50]:
compose_stadium_capacity_mapping.head()

,venue,capacity
0,国立霞ヶ丘競技場,48000.0
3,県立カシマサッカースタジアム,40728.0
4,万博記念競技場,21000.0
5,日本平運動公園球技場,20339.0
6,浦和市駒場スタジアム,21500.0


In [51]:
compose_processing.head()

,index,attendance,away_team,broadcasters,home_team,humidity,id,kick_off_time,match_date,round,section,temperature,venue,weather
0,0,59626.0,0,0,0,60.653314,49,19:29,1993-05-15,0,0,20.943044,国立霞ヶ丘競技場,晴
1,1,14126.0,1,0,1,60.653314,50,13:05,1993-05-16,1,0,20.943044,横浜市三ツ沢総合公園球技場,晴
2,2,11875.0,2,0,2,60.653314,51,13:59,1993-05-16,1,0,20.943044,広島スタジアム,晴
3,3,10898.0,3,0,3,60.653314,52,16:00,1993-05-16,1,0,20.943044,県立カシマサッカースタジアム,晴
4,4,19580.0,4,0,4,60.653314,53,19:04,1993-05-16,1,0,20.943044,万博記念競技場,晴


In [52]:
#mergeする
compose_processing = pd.merge(compose_processing,compose_stadium_capacity_mapping, on =["venue"],how="left")

In [53]:
print(compose_processing.shape)

(9010, 15)


In [54]:
#スタジアムの辞書の作成
venue_dict={}
add_venue_dict ={}
for venue_num in range(venue_number):
    venue_dict.update({compose_processing["venue"].drop_duplicates().unique()[venue_num]:venue_num})
    add_venue_dict.update({compose_processing["venue"].drop_duplicates().unique()[venue_num]:venue_num})

################################################################################################################
#置換
#stadium_capacity_mapping["stadium"] = stadium_capacity_mapping["stadium"].replace(add_venue_dict)
compose_processing["venue"] =compose_processing["venue"].replace(venue_dict)
########################################################################################################

In [55]:
#天気の辞書の作成
weather_dict={}
for weather_num in range (weather_number):
    weather_dict.update({compose["weather"].drop_duplicates().unique()[weather_num]:weather_num})
#置換
compose_processing["weather"] =compose_processing["weather"].replace(weather_dict)

In [56]:
weather_dict

{'晴': 0,
 '曇': 1,
 '雨': 2,
 '曇時々雨': 3,
 '曇一時晴': 4,
 '曇のち雨': 5,
 '曇のち晴': 6,
 '晴のち曇': 7,
 '曇一時雨': 8,
 '雨のち曇': 9,
 '曇時々晴': 10,
 '晴のち雨': 11,
 '晴時々曇': 12,
 '晴時々雪': 13,
 '晴一時雨': 14,
 '雨時々曇': 15,
 '雷雨': 16,
 '雨一時曇': 17,
 '雨のち晴': 18,
 '曇のち雷雨': 19,
 '晴一時曇': 20,
 '曇一時雨のち曇': 21,
 '曇一時雷雨': 22,
 '晴のち雷雨': 23,
 '雪時々雨': 24,
 '雨時々雪': 25,
 '晴時々雨': 26,
 '屋内': 27,
 '晴のち曇のち雨': 28,
 '晴のち霧': 29,
 '霧': 30,
 '霧のち曇': 31,
 '曇のち雨のち曇': 32,
 '曇のち雨のち屋内': 33,
 '雪一時曇': 34,
 '曇時々雪一時晴': 35,
 '雪': 36,
 '雨一時霧': 37,
 '曇のち雷雨のち曇': 38,
 '晴のち雷雨のち曇': 39,
 '曇時々雪時々晴': 40,
 '晴のち雷のち雨': 41,
 '雨一時曇のち雨': 42,
 '曇のち晴一時雨': 43,
 '雨時々晴': 44,
 '晴のち曇のち雷雨': 45,
 '雨のち曇のち雨': 46,
 '曇のち雪': 47,
 '晴のち曇一時雨': 48,
 '雪のち雨': 49,
 '雷雨のち雨': 50,
 '曇晴': 51,
 '曇一時雪': 52,
 '曇のち雨のち晴': 53,
 '曇時々晴のち屋内': 54,
 '雨のち曇時々雨': 55,
 '曇時々雪': 56,
 '曇のち雨一時雷': 57,
 '晴時々曇一時雨': 58,
 '雷雨のち曇': 59,
 '曇一時雷雨のち雨': 60,
 '晴一時雨のち曇': 61,
 '雨のち曇のち晴': 62,
 '曇一時雨のち晴': 63,
 '曇一時雷雨のち曇': 64,
 '曇時々雨のち晴': 65,
 '曇時々晴一時雨': 66,
 '雨のち曇一時雨': 67,
 '晴のち雪': 68,
 '雨一時雷雨': 69,
 '曇のち霧': 70,
 '曇時々雨のち曇': 71

In [57]:
#nanデータ　平均で置換
compose_processing["weather"]   = compose_processing["weather"].fillna(compose_processing["weather"].mean())

## 休日の追加

In [58]:
import datetime

In [59]:
compose_processing["match_date"] = pd.to_datetime(compose_processing["match_date"])
compose_processing["kick_off_time"] = pd.to_datetime(compose_processing["kick_off_time"])

In [60]:
#compose_processingにholidayを追加するため
#mergeするために,列名を同じ名前にする .rename
holidays_in_japan  = holidays_in_japan .rename(columns={"holiday_date":"match_date"})
holidays_in_japan["match_date"] = pd.to_datetime(holidays_in_japan["match_date"])

In [61]:
compose_processing=pd.merge(compose_processing,holidays_in_japan ,how = "left" , on = "match_date")

In [62]:
holidays_in_japan

,match_date,description
0,2006-01-01,New Years Day
1,2006-01-02,New Years Holiday
2,2006-01-03,New Years Holiday
3,2006-01-09,Coming-of-age Day
4,2006-02-11,National Foundation Day
5,2006-03-21,Vernal Equinox Day
6,2006-04-29,Greenery Day
7,2006-05-03,Constitution Memorial Day
8,2006-05-04,National Holiday
9,2006-05-05,Childrens Day


In [63]:
#ユニーク数カウント
description_number =compose_processing["description"].drop_duplicates().nunique()

In [64]:
#休日の辞書作成
holiday_dict={}
for description_num in range(description_number+1):
    holiday_dict.update({compose_processing["description"].drop_duplicates().unique()[description_num]:description_num})
#置換
compose_processing["description"] =compose_processing["description"].replace(holiday_dict)

In [65]:
holiday_dict

{nan: 0,
 'Vernal Equinox Day': 1,
 'Greenery Day': 2,
 'Constitution Memorial Day': 3,
 'Autumnal Equinox Day': 4,
 'Labor Thanksgiving Day': 5,
 'Showa Day': 6,
 'National Holiday': 7,
 'Marine Day': 8,
 'Childrens Day': 9,
 'Culture Day': 10,
 'Childrens Day (obs)': 11,
 'Greenery Day (obs)': 12}

In [66]:
compose_processing

,index,attendance,away_team,broadcasters,home_team,humidity,id,kick_off_time,match_date,round,section,temperature,venue,weather,capacity,description
0,0,59626.0,0,0,0,60.653314,49,2018-11-10 19:29:00,1993-05-15,0,0,20.943044,0,0,48000.0,0
1,1,14126.0,1,0,1,60.653314,50,2018-11-10 13:05:00,1993-05-16,1,0,20.943044,1,0,NaN,0
2,2,11875.0,2,0,2,60.653314,51,2018-11-10 13:59:00,1993-05-16,1,0,20.943044,2,0,NaN,0
3,3,10898.0,3,0,3,60.653314,52,2018-11-10 16:00:00,1993-05-16,1,0,20.943044,3,0,40728.0,0
4,4,19580.0,4,0,4,60.653314,53,2018-11-10 19:04:00,1993-05-16,1,0,20.943044,4,0,21000.0,0
5,5,9183.0,5,0,5,60.653314,58,2018-11-10 18:58:00,1993-05-19,0,1,20.943044,5,1,20339.0,0
6,6,9259.0,3,0,6,60.653314,56,2018-11-10 19:00:00,1993-05-19,0,1,20.943044,6,0,21500.0,0
7,7,10332.0,6,0,7,60.653314,57,2018-11-10 19:00:00,1993-05-19,0,1,20.943044,1,0,NaN,0
8,8,9403.0,7,0,3,60.653314,54,2018-11-10 19:01:00,1993-05-19,0,1,20.943044,3,0,40728.0,0
9,9,46959.0,8,0,8,60.653314,55,2018-11-10 19:02:00,1993-05-19,0,1,20.943044,0,0,48000.0,0


### 時間についてまとめる

In [67]:
####################################
## 列の追加
compose_processing["match_date_year"] = 0
compose_processing["match_date_month"] = 0
compose_processing["match_date_day"] = 0
################################
compose_processing["match_date_year"]  = compose_processing["match_date"].dt.year
compose_processing["match_date_month"] = compose_processing["match_date"].dt.month
compose_processing["match_date_day"]   = compose_processing["match_date"].dt.day

In [68]:
#キックオフの時間の追加
compose_processing["kick_off_time_hour"] =0
compose_processing["kick_off_time_hour"] = compose_processing["kick_off_time"].dt.hour
#######################################################################################
compose_processing["kick_off_time_minute"] =0
compose_processing["kick_off_time_minute"] =compose_processing["kick_off_time"].dt.minute

In [69]:
#曜日の追加
compose_processing["match_date_weekday"] = 0
compose_processing["match_date_weekday"] = compose_processing["match_date"].dt.dayofweek
############################################################################

In [70]:
#kick-off_timeのデータは使わないので削除する
compose_processing = compose_processing.drop("kick_off_time", axis =1)

In [71]:
compose_processing

,index,attendance,away_team,broadcasters,home_team,humidity,id,match_date,round,section,...,venue,weather,capacity,description,match_date_year,match_date_month,match_date_day,kick_off_time_hour,kick_off_time_minute,match_date_weekday
0,0,59626.0,0,0,0,60.653314,49,1993-05-15,0,0,...,0,0,48000.0,0,1993,5,15,19,29,5
1,1,14126.0,1,0,1,60.653314,50,1993-05-16,1,0,...,1,0,NaN,0,1993,5,16,13,5,6
2,2,11875.0,2,0,2,60.653314,51,1993-05-16,1,0,...,2,0,NaN,0,1993,5,16,13,59,6
3,3,10898.0,3,0,3,60.653314,52,1993-05-16,1,0,...,3,0,40728.0,0,1993,5,16,16,0,6
4,4,19580.0,4,0,4,60.653314,53,1993-05-16,1,0,...,4,0,21000.0,0,1993,5,16,19,4,6
5,5,9183.0,5,0,5,60.653314,58,1993-05-19,0,1,...,5,1,20339.0,0,1993,5,19,18,58,2
6,6,9259.0,3,0,6,60.653314,56,1993-05-19,0,1,...,6,0,21500.0,0,1993,5,19,19,0,2
7,7,10332.0,6,0,7,60.653314,57,1993-05-19,0,1,...,1,0,NaN,0,1993,5,19,19,0,2
8,8,9403.0,7,0,3,60.653314,54,1993-05-19,0,1,...,3,0,40728.0,0,1993,5,19,19,1,2
9,9,46959.0,8,0,8,60.653314,55,1993-05-19,0,1,...,0,0,48000.0,0,1993,5,19,19,2,2


### 点数と勝敗だけいれる　外部データと統合

In [74]:
type(compose_match_reports2["id"][10])

str

In [75]:
compose_processing1 = compose_processing.drop("index",axis=1)

In [76]:
compose_processing1.shape

(9010, 20)

In [77]:
compose_processing2 = pd.merge(compose_processing1 , compose_match_reports2 , on="id" ,how="left")

In [78]:
compose_match_reports2.shape

(14376, 4)

In [79]:
compose_match_reports2.head(500)

,away_team_score,home_team_score,home_team_win,id
0,2,1,False,49
1,2,3,True,50
2,1,2,True,51
3,0,5,True,52
4,0,1,True,53
5,0,1,True,58
6,3,0,False,56
7,0,1,True,57
8,2,3,True,54
9,1,2,True,55


In [80]:
compose_processing2.head(500)

,attendance,away_team,broadcasters,home_team,humidity,id,match_date,round,section,temperature,...,description,match_date_year,match_date_month,match_date_day,kick_off_time_hour,kick_off_time_minute,match_date_weekday,away_team_score,home_team_score,home_team_win
0,59626.0,0,0,0,60.653314,49,1993-05-15,0,0,20.943044,...,0,1993,5,15,19,29,5,2.0,1.0,False
1,14126.0,1,0,1,60.653314,50,1993-05-16,1,0,20.943044,...,0,1993,5,16,13,5,6,2.0,3.0,True
2,11875.0,2,0,2,60.653314,51,1993-05-16,1,0,20.943044,...,0,1993,5,16,13,59,6,1.0,2.0,True
3,10898.0,3,0,3,60.653314,52,1993-05-16,1,0,20.943044,...,0,1993,5,16,16,0,6,0.0,5.0,True
4,19580.0,4,0,4,60.653314,53,1993-05-16,1,0,20.943044,...,0,1993,5,16,19,4,6,0.0,1.0,True
5,9183.0,5,0,5,60.653314,58,1993-05-19,0,1,20.943044,...,0,1993,5,19,18,58,2,0.0,1.0,True
6,9259.0,3,0,6,60.653314,56,1993-05-19,0,1,20.943044,...,0,1993,5,19,19,0,2,3.0,0.0,False
7,10332.0,6,0,7,60.653314,57,1993-05-19,0,1,20.943044,...,0,1993,5,19,19,0,2,0.0,1.0,True
8,9403.0,7,0,3,60.653314,54,1993-05-19,0,1,20.943044,...,0,1993,5,19,19,1,2,2.0,3.0,True
9,46959.0,8,0,8,60.653314,55,1993-05-19,0,1,20.943044,...,0,1993,5,19,19,2,2,1.0,2.0,True


In [81]:
print(compose_processing1.shape)
print(compose_processing2.shape)

(9010, 20)
(9010, 23)


In [82]:
compose_processing2

,attendance,away_team,broadcasters,home_team,humidity,id,match_date,round,section,temperature,...,description,match_date_year,match_date_month,match_date_day,kick_off_time_hour,kick_off_time_minute,match_date_weekday,away_team_score,home_team_score,home_team_win
0,59626.0,0,0,0,60.653314,49,1993-05-15,0,0,20.943044,...,0,1993,5,15,19,29,5,2.0,1.0,False
1,14126.0,1,0,1,60.653314,50,1993-05-16,1,0,20.943044,...,0,1993,5,16,13,5,6,2.0,3.0,True
2,11875.0,2,0,2,60.653314,51,1993-05-16,1,0,20.943044,...,0,1993,5,16,13,59,6,1.0,2.0,True
3,10898.0,3,0,3,60.653314,52,1993-05-16,1,0,20.943044,...,0,1993,5,16,16,0,6,0.0,5.0,True
4,19580.0,4,0,4,60.653314,53,1993-05-16,1,0,20.943044,...,0,1993,5,16,19,4,6,0.0,1.0,True
5,9183.0,5,0,5,60.653314,58,1993-05-19,0,1,20.943044,...,0,1993,5,19,18,58,2,0.0,1.0,True
6,9259.0,3,0,6,60.653314,56,1993-05-19,0,1,20.943044,...,0,1993,5,19,19,0,2,3.0,0.0,False
7,10332.0,6,0,7,60.653314,57,1993-05-19,0,1,20.943044,...,0,1993,5,19,19,0,2,0.0,1.0,True
8,9403.0,7,0,3,60.653314,54,1993-05-19,0,1,20.943044,...,0,1993,5,19,19,1,2,2.0,3.0,True
9,46959.0,8,0,8,60.653314,55,1993-05-19,0,1,20.943044,...,0,1993,5,19,19,2,2,1.0,2.0,True


おかしい　数が合わない…

In [83]:
print(compose_processing1["id"].value_counts())
print(len(compose_processing1["id"].value_counts()))

90       1
7124     1
11839    1
7521     1
5518     1
546      1
6569     1
7149     1
2966     1
7113     1
4669     1
10858    1
8056     1
11671    1
592      1
6731     1
3716     1
5495     1
5709     1
5999     1
5398     1
7110     1
7211     1
3899     1
2165     1
8179     1
3019     1
11664    1
5179     1
9498     1
        ..
11648    1
1502     1
11646    1
11645    1
11643    1
11668    1
7393     1
11641    1
11640    1
11639    1
11638    1
11637    1
11651    1
11652    1
11653    1
11654    1
11655    1
11656    1
11657    1
2943     1
11659    1
11660    1
11661    1
8669     1
11663    1
7441     1
11665    1
11666    1
11667    1
8941     1
Name: id, Length: 9010, dtype: int64
9010


In [84]:
print(compose_match_reports["id"].value_counts())
print(len(compose_match_reports["id"].value_counts()))

11070    2
14268    2
11686    2
15780    2
14284    2
10190    2
11670    2
15764    2
10206    2
11654    2
15748    2
10222    2
11638    2
15732    2
10238    2
11622    2
15716    2
10254    2
15700    2
10174    2
15796    2
9263     2
11702    2
11766    2
14204    2
10110    2
15860    2
17899    2
11750    2
14220    2
        ..
3784     1
16062    1
11964    1
5819     1
7866     1
3768     1
3832     1
7930     1
5883     1
12028    1
3896     1
18225    1
16174    1
12076    1
5931     1
7978     1
3880     1
18209    1
16158    1
12060    1
5915     1
7962     1
3864     1
16142    1
12044    1
5899     1
7946     1
3848     1
16126    1
2049     1
Name: id, Length: 14376, dtype: int64
14376


まさかのmatch_reportsに重複アリ

In [85]:
#重複行削除
compose_match_reports.drop_duplicates(inplace = True,subset="id")

In [86]:
compose_match_reports.shape

(14376, 4)

In [87]:
compose_processing3 = compose_processing1.merge(compose_match_reports2[["id", "home_team_win", "home_team_score", "away_team_score"]] , how="left",
                          on="id")

In [88]:
compose_processing3.shape 

(9010, 23)

In [89]:
compose_processing3.head(500)

,attendance,away_team,broadcasters,home_team,humidity,id,match_date,round,section,temperature,...,description,match_date_year,match_date_month,match_date_day,kick_off_time_hour,kick_off_time_minute,match_date_weekday,home_team_win,home_team_score,away_team_score
0,59626.0,0,0,0,60.653314,49,1993-05-15,0,0,20.943044,...,0,1993,5,15,19,29,5,False,1.0,2.0
1,14126.0,1,0,1,60.653314,50,1993-05-16,1,0,20.943044,...,0,1993,5,16,13,5,6,True,3.0,2.0
2,11875.0,2,0,2,60.653314,51,1993-05-16,1,0,20.943044,...,0,1993,5,16,13,59,6,True,2.0,1.0
3,10898.0,3,0,3,60.653314,52,1993-05-16,1,0,20.943044,...,0,1993,5,16,16,0,6,True,5.0,0.0
4,19580.0,4,0,4,60.653314,53,1993-05-16,1,0,20.943044,...,0,1993,5,16,19,4,6,True,1.0,0.0
5,9183.0,5,0,5,60.653314,58,1993-05-19,0,1,20.943044,...,0,1993,5,19,18,58,2,True,1.0,0.0
6,9259.0,3,0,6,60.653314,56,1993-05-19,0,1,20.943044,...,0,1993,5,19,19,0,2,False,0.0,3.0
7,10332.0,6,0,7,60.653314,57,1993-05-19,0,1,20.943044,...,0,1993,5,19,19,0,2,True,1.0,0.0
8,9403.0,7,0,3,60.653314,54,1993-05-19,0,1,20.943044,...,0,1993,5,19,19,1,2,True,3.0,2.0
9,46959.0,8,0,8,60.653314,55,1993-05-19,0,1,20.943044,...,0,1993,5,19,19,2,2,True,2.0,1.0


In [90]:
#重複行削除
compose_processing4= compose_processing3.drop_duplicates(inplace = False,subset="id")

In [91]:
compose_processing4.shape 

(9010, 23)

数がようやくあった

In [92]:
compose_processing3

,attendance,away_team,broadcasters,home_team,humidity,id,match_date,round,section,temperature,...,description,match_date_year,match_date_month,match_date_day,kick_off_time_hour,kick_off_time_minute,match_date_weekday,home_team_win,home_team_score,away_team_score
0,59626.0,0,0,0,60.653314,49,1993-05-15,0,0,20.943044,...,0,1993,5,15,19,29,5,False,1.0,2.0
1,14126.0,1,0,1,60.653314,50,1993-05-16,1,0,20.943044,...,0,1993,5,16,13,5,6,True,3.0,2.0
2,11875.0,2,0,2,60.653314,51,1993-05-16,1,0,20.943044,...,0,1993,5,16,13,59,6,True,2.0,1.0
3,10898.0,3,0,3,60.653314,52,1993-05-16,1,0,20.943044,...,0,1993,5,16,16,0,6,True,5.0,0.0
4,19580.0,4,0,4,60.653314,53,1993-05-16,1,0,20.943044,...,0,1993,5,16,19,4,6,True,1.0,0.0
5,9183.0,5,0,5,60.653314,58,1993-05-19,0,1,20.943044,...,0,1993,5,19,18,58,2,True,1.0,0.0
6,9259.0,3,0,6,60.653314,56,1993-05-19,0,1,20.943044,...,0,1993,5,19,19,0,2,False,0.0,3.0
7,10332.0,6,0,7,60.653314,57,1993-05-19,0,1,20.943044,...,0,1993,5,19,19,0,2,True,1.0,0.0
8,9403.0,7,0,3,60.653314,54,1993-05-19,0,1,20.943044,...,0,1993,5,19,19,1,2,True,3.0,2.0
9,46959.0,8,0,8,60.653314,55,1993-05-19,0,1,20.943044,...,0,1993,5,19,19,2,2,True,2.0,1.0


In [93]:
compose_processing2 = compose_processing2.drop("home_team_score_x", axis =1)
compose_processing2 = compose_processing2.drop("away_team_score_x", axis =1)
compose_processing2 = compose_processing2.drop("home_team_win_x", axis =1)

ValueError: labels ['home_team_score_x'] not contained in axis

In [94]:
compose_processing2 = compose_processing2.rename(columns={"home_team_score_y":"home_team_score"})
compose_processing2 = compose_processing2.rename(columns={"away_team_score_y":"away_team_score"})
compose_processing2 = compose_processing2.rename(columns={"home_team_win_y":"home_team_win"})

### データをソート　match-dateでソートした

In [95]:
compose_processing4.sort_values(by = "match_date")
compose_processing4.reset_index

<bound method DataFrame.reset_index of       attendance  away_team  broadcasters  home_team   humidity     id  \
0        59626.0          0             0          0  60.653314     49   
1        14126.0          1             0          1  60.653314     50   
2        11875.0          2             0          2  60.653314     51   
3        10898.0          3             0          3  60.653314     52   
4        19580.0          4             0          4  60.653314     53   
5         9183.0          5             0          5  60.653314     58   
6         9259.0          3             0          6  60.653314     56   
7        10332.0          6             0          7  60.653314     57   
8         9403.0          7             0          3  60.653314     54   
9        46959.0          8             0          8  60.653314     55   
10       11267.0          0             0          9  60.653314     59   
11       36863.0          8             0          2  60.653314     61   

In [96]:
compose_processing4

,attendance,away_team,broadcasters,home_team,humidity,id,match_date,round,section,temperature,...,description,match_date_year,match_date_month,match_date_day,kick_off_time_hour,kick_off_time_minute,match_date_weekday,home_team_win,home_team_score,away_team_score
0,59626.0,0,0,0,60.653314,49,1993-05-15,0,0,20.943044,...,0,1993,5,15,19,29,5,False,1.0,2.0
1,14126.0,1,0,1,60.653314,50,1993-05-16,1,0,20.943044,...,0,1993,5,16,13,5,6,True,3.0,2.0
2,11875.0,2,0,2,60.653314,51,1993-05-16,1,0,20.943044,...,0,1993,5,16,13,59,6,True,2.0,1.0
3,10898.0,3,0,3,60.653314,52,1993-05-16,1,0,20.943044,...,0,1993,5,16,16,0,6,True,5.0,0.0
4,19580.0,4,0,4,60.653314,53,1993-05-16,1,0,20.943044,...,0,1993,5,16,19,4,6,True,1.0,0.0
5,9183.0,5,0,5,60.653314,58,1993-05-19,0,1,20.943044,...,0,1993,5,19,18,58,2,True,1.0,0.0
6,9259.0,3,0,6,60.653314,56,1993-05-19,0,1,20.943044,...,0,1993,5,19,19,0,2,False,0.0,3.0
7,10332.0,6,0,7,60.653314,57,1993-05-19,0,1,20.943044,...,0,1993,5,19,19,0,2,True,1.0,0.0
8,9403.0,7,0,3,60.653314,54,1993-05-19,0,1,20.943044,...,0,1993,5,19,19,1,2,True,3.0,2.0
9,46959.0,8,0,8,60.653314,55,1993-05-19,0,1,20.943044,...,0,1993,5,19,19,2,2,True,2.0,1.0


In [97]:
print(compose_processing3.match_date)

0      1993-05-15
1      1993-05-16
2      1993-05-16
3      1993-05-16
4      1993-05-16
5      1993-05-19
6      1993-05-19
7      1993-05-19
8      1993-05-19
9      1993-05-19
10     1993-05-22
11     1993-05-22
12     1993-05-22
13     1993-05-22
14     1993-05-22
15     1993-05-26
16     1993-05-26
17     1993-05-26
18     1993-05-26
19     1993-05-26
20     1993-05-29
21     1993-05-29
22     1993-05-29
23     1993-05-29
24     1993-05-29
25     1993-06-02
26     1993-06-02
27     1993-06-02
28     1993-06-02
29     1993-06-02
          ...    
8980   2018-07-18
8981   2018-07-18
8982   2018-07-18
8983   2018-07-22
8984   2018-07-22
8985   2018-07-22
8986   2018-07-22
8987   2018-07-22
8988   2018-07-22
8989   2018-07-22
8990   2018-07-22
8991   2018-07-22
8992   2018-11-24
8993   2018-11-24
8994   2018-11-24
8995   2018-11-24
8996   2018-11-24
8997   2018-11-24
8998   2018-11-24
8999   2018-11-24
9000   2018-11-24
9001   2018-12-01
9002   2018-12-01
9003   2018-12-01
9004   201

In [98]:
compose_processing.to_csv("compose_processing.csv",header=True,index=None)

In [99]:
compose_processing2.to_csv("compose_processing2.csv",header=True,index=None)
compose_processing3.to_csv("compose_processing3.csv",header=True,index=None)
compose_processing4.to_csv("compose_processing4.csv",header=True,index=None)

# 工事中　one-hot-vector

In [100]:
pd.set_option('display.max_columns', 100)

# 天気をone-hot-vectorする

参考URL：  
https://www.haya-programming.com/entry/2018/06/09/170504

In [103]:
from sklearn.preprocessing import OneHotEncoder

In [104]:
weather_vector=OneHotEncoder().fit_transform(compose_processing4["weather"].reshape(-1, 1)).A

C:\Users\Mustang8\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead



In [105]:
weather_vector.shape

(9010, 73)

In [106]:
weather_vector = pd.DataFrame(weather_vector)
weather_vector = weather_vector.rename(columns=lambda x: "weather"+ x.astype(str))

In [107]:
compose_processing5 = pd.concat([compose_processing4,weather_vector],axis=1 )

In [108]:
compose_processing5

,attendance,away_team,broadcasters,home_team,humidity,id,match_date,round,section,temperature,venue,weather,capacity,description,match_date_year,match_date_month,match_date_day,kick_off_time_hour,kick_off_time_minute,match_date_weekday,home_team_win,home_team_score,away_team_score,weather0,weather1,weather2,weather3,weather4,weather5,weather6,weather7,weather8,weather9,weather10,weather11,weather12,weather13,weather14,weather15,weather16,weather17,weather18,weather19,weather20,weather21,weather22,weather23,weather24,weather25,weather26,weather27,weather28,weather29,weather30,weather31,weather32,weather33,weather34,weather35,weather36,weather37,weather38,weather39,weather40,weather41,weather42,weather43,weather44,weather45,weather46,weather47,weather48,weather49,weather50,weather51,weather52,weather53,weather54,weather55,weather56,weather57,weather58,weather59,weather60,weather61,weather62,weather63,weather64,weather65,weather66,weather67,weather68,weather69,weather70,weather71,weather72
0,59626.0,0,0,0,60.653314,49,1993-05-15,0,0,20.943044,0,0,48000.0,0,1993,5,15,19,29,5,False,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,14126.0,1,0,1,60.653314,50,1993-05-16,1,0,20.943044,1,0,NaN,0,1993,5,16,13,5,6,True,3.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,11875.0,2,0,2,60.653314,51,1993-05-16,1,0,20.943044,2,0,NaN,0,1993,5,16,13,59,6,True,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10898.0,3,0,3,60.653314,52,1993-05-16,1,0,20.943044,3,0,40728.0,0,1993,5,16,16,0,6,True,5.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,19580.0,4,0,4,60.653314,53,1993-05-16,1,0,20.943044,4,0,21000.0,0,1993,5,16,19,4,6,True,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,9183.0,5,0,5,60.653314,58,1993-05-19,0,1,20.943044,5,1,20339.0,0,1993,5,19,18,58,2,True,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,9259.0,3,0,6,60.653314,56,1993-05-19,0,1,20.943044,6,0,21500.0,0,1993,5,19,19,0,2,False,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,10332.0,6,0,7,60.653314,57,1993-05-19,0,1,20.943044,1,0,NaN,0,1993,5,19,19,0,2,True,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [109]:
compose_processing5.to_csv("compose_processing5.csv",header=True,index=None)

# スタジアムをone-hot-vector

In [110]:
venue_vector=OneHotEncoder().fit_transform(compose_processing4["venue"].reshape(-1, 1)).A

C:\Users\Mustang8\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead



In [111]:
venue_vector = pd.DataFrame(venue_vector)
venue_vector = venue_vector.rename(columns=lambda x: "venuer"+ x.astype(str))

In [112]:
compose_processing6 = pd.concat([compose_processing5,venue_vector],axis=1 )

In [113]:
compose_processing6.to_csv("compose_processing6.csv",header=True,index=None)

In [114]:
compose_processing6

,attendance,away_team,broadcasters,home_team,humidity,id,match_date,round,section,temperature,venue,weather,capacity,description,match_date_year,match_date_month,match_date_day,kick_off_time_hour,kick_off_time_minute,match_date_weekday,home_team_win,home_team_score,away_team_score,weather0,weather1,weather2,weather3,weather4,weather5,weather6,weather7,weather8,weather9,weather10,weather11,weather12,weather13,weather14,weather15,weather16,weather17,weather18,weather19,weather20,weather21,weather22,weather23,weather24,weather25,weather26,...,venuer98,venuer99,venuer100,venuer101,venuer102,venuer103,venuer104,venuer105,venuer106,venuer107,venuer108,venuer109,venuer110,venuer111,venuer112,venuer113,venuer114,venuer115,venuer116,venuer117,venuer118,venuer119,venuer120,venuer121,venuer122,venuer123,venuer124,venuer125,venuer126,venuer127,venuer128,venuer129,venuer130,venuer131,venuer132,venuer133,venuer134,venuer135,venuer136,venuer137,venuer138,venuer139,venuer140,venuer141,venuer142,venuer143,venuer144,venuer145,venuer146,venuer147
0,59626.0,0,0,0,60.653314,49,1993-05-15,0,0,20.943044,0,0,48000.0,0,1993,5,15,19,29,5,False,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,14126.0,1,0,1,60.653314,50,1993-05-16,1,0,20.943044,1,0,NaN,0,1993,5,16,13,5,6,True,3.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,11875.0,2,0,2,60.653314,51,1993-05-16,1,0,20.943044,2,0,NaN,0,1993,5,16,13,59,6,True,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10898.0,3,0,3,60.653314,52,1993-05-16,1,0,20.943044,3,0,40728.0,0,1993,5,16,16,0,6,True,5.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,19580.0,4,0,4,60.653314,53,1993-05-16,1,0,20.943044,4,0,21000.0,0,1993,5,16,19,4,6,True,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,9183.0,5,0,5,60.653314,58,1993-05-19,0,1,20.943044,5,1,20339.0,0,1993,5,19,18,58,2,True,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,9259.0,3,0,6,60.653314,56,1993-05-19,0,1,20.943044,6,0,21500.0,0,1993,5,19,19,0,2,False,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,10332.0,6,0,7,60.653314,57,1993-05-19,0,1,20.943044,1,0,NaN,0,1993,5,19,19,0,2,True,1.0,0

参考  
https://qiita.com/uratatsu/items/8bedbf91e22f90b6e64b  
http://sinhrks.hatenablog.com/entry/2015/01/28/073327  
https://note.nkmk.me/python-pandas-merge-join/  
